# Initialize data, load packages and needed modules

In [1]:
random_seeds = [42, 100, 0, 10, 12, 20, 52, 9, 30, 51]
run = 0

In [2]:
username = 'meganorm-yverduyn'
datasets = {
    'TDBrain': {''
        'base_dir': '/project/meganorm/Data/EEG_TDBrain/EEG/', #Final
        #'base_dir': '/home/meganorm-yverduyn/Dev/BIDS_TDBrain',  #To test 
        'task': 'task-restEC', 
        'ending': 'eeg.vhdr'
    },

    'MIPDB': {
        'base_dir': '/project/meganorm/Data/EEG_MIPDB/EEG_BIDS/', #Final
        #'base_dir': '/home/meganorm-yverduyn/Dev/MIPDB/EEG_BIDS',  #To test 
        'task': 'task-eyesclosed', 
        'ending': 'eeg.set'
},

    'CMI': {
        'base_dir': '/project/meganorm/Data/EEG_CMI/EEG_BIDS/', #Final
        #'base_dir': '/home/meganorm-yverduyn/Dev/BIDS_CMI',  #To test 
        'task': 'task-eyesclosed', 
        'ending': 'eeg.set'
    }
    }

package_path = f'/home/{username}/Code/MEGaNorm/'

In [ ]:
#%pip install https://github.com/amarquand/PCNtoolkit/archive/refs/tags/v1.alpha.zip
#%pip install graphviz
# switch to refactor branch: 
# *  git clone --branch dev_refactor https://github.com/amarquand/PCNtoolkit.git
# *  git checkout dev_refactor
# *  python -m pip install .

In [13]:
import os

from pcntoolkit.dataio.norm_data import NormData
from pcntoolkit.regression_model.hbr import make_prior, HBR, SHASHbLikelihood
from pcntoolkit.math.basis_function import BsplineBasisFunction
from pcntoolkit.normative_model import NormativeModel
from pcntoolkit.util.runner import Runner

os.chdir(package_path)
import pandas as pd
import json
import numpy as np
import pickle 
import warnings

from utils.IO import merge_datasets_with_glob, merge_fidp_demo
from utils.parallel import submit_jobs, check_jobs_status, collect_results
from datasets.mne_bids_conversion import make_demo_file_bids
from plots.plots import plot_age_dist2

warnings.filterwarnings("ignore")

# Create configuration file

In [4]:
def make_config(project, path=None):

    # preprocess configurations =================================================
    # downsample data
    config = dict()

    # You could also set layout to None to have high 
    # choices: all, lobe, None
    config["which_layout"] = "all"

    # which sensor type should be used
    # choices: meg, mag, grad, eeg, opm
    config["which_sensor"] = "eeg"
    # config['fs'] = 500

    # ICA configuration
    config['ica_n_component'] = 25
    config['ica_max_iter'] = 800
    config['ica_method'] = "infomax" 

    # lower and upper cutoff frequencies in a bandpass filter
    config['cutoffFreqLow'] = 1
    config['cutoffFreqHigh'] = 45

    config["resampling_rate"] = 1000
    config["digital_filter"] = True
    config["notch_filter"] = False

    config["apply_ica"] = True

    config["auto_ica_corr_thr"] = 0.8

    config["rereference_method"]= "average"
    
    # variance threshold across time
    config["mag_var_threshold"] = 4e-12
    config["grad_var_threshold"] = 4000e-13
    config["eeg_var_threshold"] = 40e-6  
    # flatness threshold across time
    config["mag_flat_threshold"] = 10e-15
    config["grad_flat_threshold"] = 10e-15
    config["eeg_flat_threshold"] = 10e-6 

    # segmentation ==============================================
    #start time of the raw data to use in seconds, this is to avoid possible eye blinks in close-eyed resting state. 
    config['segments_tmin'] = 5
    # end time of the raw data to use in seconds, this is to avoid possible eye blinks in close-eyed resting state.
    config['segments_tmax'] = -5
    # length of EEG segments in seconds
    config['segments_length'] = 10
    # amount of overlap between EEG sigals in seconds
    config['segments_overlap'] = 2

    # PSD ==============================================
    # Spectral estimation method
    config['psd_method'] = "welch"
    # amount of overlap between windows in Welch's method
    config['psd_n_overlap'] = 1
    config['psd_n_fft'] = 2
    # number of samples in psd
    config["psd_n_per_seg"] = 2

    # fooof analysis configurations ==============================================
    # Desired frequency range to run FOOOF
    config['fooof_freqRangeLow'] = 3
    config['fooof_freqRangeHigh'] = 40
    # which mode should be used for fitting; choices (knee, fixed)
    config["aperiodic_mode"] = "knee"
    # minimum acceptable peak width in fooof analysis
    config["fooof_peak_width_limits"] = [1.0, 12.0]
    #Absolute threshold for detecting peaks
    config['fooof_min_peak_height'] = 0
    #Relative threshold for detecting peaks
    config['fooof_peak_threshold'] = 2

    # feature extraction ==========================================================
    # Define frequency bands
    config['freq_bands'] = {
                            'Theta': (3, 8),
                            'Alpha': (8, 13),
                            'Beta': (13, 30),
                            'Gamma': (30, 40),
                            # 'Broadband': (3, 40)
                            }

    # Define individualized frequency range over main peaks in each freq band
    config['individualized_band_ranges'] = { 
                                            'Theta': (-2, 3),
                                            'Alpha': (-2, 3), # change to (-4,2)
                                            'Beta': (-8, 9),
                                            'Gamma': (-5, 5)
                                            }

    # least acceptable R squred of fitted models
    config['min_r_squared'] = 0.9 
 
    config['feature_categories'] = {
                                    "Offset":False,
                                    "Exponent":False,
                                    "Peak_Center":False,
                                    "Peak_Power":False,
                                    "Peak_Width":False,
                                    "Adjusted_Canonical_Relative_Power":True, 
                                    "Adjusted_Canonical_Absolute_Power":False,
                                    "Adjusted_Individualized_Relative_Power":False,
                                    "Adjusted_Individualized_Absolute_Power":False,
                                    "OriginalPSD_Canonical_Relative_Power":False, 
                                    "OriginalPSD_Canonical_Absolute_Power":False,
                                    "OriginalPSD_Individualized_Relative_Power":False,
                                    "OriginalPSD_Individualized_Absolute_Power":False,
                                    }
    
    config["fooof_res_save_path"] = None

    config["random_state"] = 97#change? 

    if path is not None:
        out_file = open(os.path.join(path, project + ".json"), "w") 
        json.dump(config, out_file, indent = 6) 
        out_file.close()

    return config

# Define directories and job specifications needed for feature extraction and NM, merge datasets

In [5]:
project = "Thesis_refactored"

project_dir = f'/home/{username}/Results/{project}/'

mainParallel_path = os.path.join(package_path, 'src', 'mainParallel.py')

features_dir = os.path.join(project_dir, 'Features')
features_log_path = os.path.join(features_dir, 'log')
features_temp_path = os.path.join(features_dir,'temp')
pics_dir = os.path.join(project_dir, "pics")

nm_processing_dir = os.path.join(project_dir, 'NM', 'Run_' + str(run))

job_configs = {'log_path':features_log_path, 'module':'mne', 'time':'1:00:00', 'memory':'20GB', 
                'partition':'normal', 'core':1, 'node':1, 'batch_file_name':'batch_job'}

if not os.path.isdir(features_log_path):
    os.makedirs(features_log_path)

if not os.path.isdir(features_temp_path):
    os.makedirs(features_temp_path)
    
if not os.path.isdir(nm_processing_dir):
    os.makedirs(nm_processing_dir)

if not os.path.isdir(pics_dir):
    os.makedirs(pics_dir)
    
configs = make_config(project, project_dir)

subjects = merge_datasets_with_glob(datasets)

# f-IDPs Extraction

In [ ]:
### Parallel feature extraction  

# # Running Jobs
start_time = submit_jobs(mainParallel_path, features_dir, subjects, 
                features_temp_path, job_configs=job_configs, config_file=os.path.join(project_dir, project+'.json'))
# Checking jobs
failed_jobs = check_jobs_status(username, start_time)

falied_subjects = {failed_job:subjects[failed_job] for failed_job in failed_jobs}

while len(failed_jobs)>0:
    # Re-running Jobs
    start_time = submit_jobs(mainParallel_path, features_dir, falied_subjects, 
                features_temp_path, job_configs=job_configs, config_file=os.path.join(project_dir, project+'.json'))
    # Checking jobs
    failed_jobs = check_jobs_status(username, start_time)

In [ ]:
collect_results(features_dir, subjects, features_temp_path, file_name='all_features')

# Create covariate files, merge covariates and features

In [ ]:
#Create participants.tsv 
file_dir1 =  "/project/meganorm/Data/EEG_CMI/info/participants_info.csv"
save_dir1 = "/project/meganorm/Data/EEG_CMI/EEG_BIDS/participants_bids.tsv"
make_demo_file_bids(file_dir1, 
                        save_dir1, 
                        0, 
                        3, 
                        {"col_name": "sex", "col_id": 2, "mapping":{0: "Male", 1: "Female"}, "single_value":None}, 
                        {"col_name": "site", "col_id": 1, "mapping":{"CMI1": "CMI1", "CMI2": "CMI2", "CMI3": "CMI3", "CMI4": "CMI4", "CMI5": "CMI5"}, "single_value":None},
                        {"col_name": "eyes", "col_id": None, "mapping": None, "single_value":"eyes_closed"},
                        {"col_name": "diagnosis", "col_id": 12, "mapping": {
                            "No info about confirmation": "no info about confirmation",
                            "Diagnosis not confirmed": "diagnosis not confirmed",
                            "dropped out": "dropped out", 
                            "Attention-Deficit/Hyperactivity Disorder": "adhd",
                            "Specific Learning Disorder": "specific learning disorder", 
                            "Schizophrenia Spectrum and other Psychotic Disorders": "schizophrenia spectrum and other psychotic disorder", 
                            "Nonadherence to Medical Treatment": "nonadherence to medical treatment", 
                            "Autism Spectrum Disorder": "asd",
                            "Gender Dysphoria": "gender dysphoria", 
                            "No Diagnosis Given: Incomplete Eval": "unknown",
                            "Obsessive Compulsive and Related Disorders": "obsessive compulsive and related disorder",
                            "Elimination Disorders": "elimination disorder",
                            "Bipolar and Related Disorders": "bipolar and related disorder",
                            "Disruptive, Impulse Control and Conduct Disorders": "disruptive, impulse control and conduct disorder",
                            "Anxiety Disorders": "anxiety disorder",
                            "No Diagnosis Given": "control", 
                            "Neurocognitive Disorders": "neurocognitive disorder", 
                            "Depressive Disorders": "depressive disorder", 
                            "Feeding and Eating Disorders": "feeding and eating disorder", 
                            "Trauma and Stressor Related Disorders": "trauma and stressor related disorder", 
                            "nan": "unknown", "Communication Disorder": "communication disorder", 
                            "Intellectual Disability": "intellectual disability",
                            "Motor Disorder": "motor disorder", 
                            "Cannabis-Related": "cannabis-related", 
                            "Substance Related and Addictive Disorders": "substance related and addictive disorders",
                            "Other Neurodevelopmental Disorders": "other neurodevelopmental disorder"}, "single_value":None})

file_dir2 = "/project/meganorm/Data/EEG_TDBrain/EEG/TDBRAIN_participants_V2.tsv"
save_dir2 = "/project/meganorm/Data/EEG_TDBrain/EEG/participants_bids.tsv"
make_demo_file_bids(file_dir2, 
                        save_dir2, 
                        0, 
                        10, 
                        {"col_name": "sex", "col_id": 11, "mapping": {1.0: "Male", 0.0: "Female"}, "single_value":None},
                        {"col_name": "eyes", "col_id": None, "mapping": None, "single_value":"eyes_closed"},
                        {"col_name": "diagnosis", "col_id": 3, "mapping": {
                            "UNKNOWN": "unknown", 
                            "REPLICATION": "replication", "BURNOUT": "burnout",  "SMC": "smc", 
                            "HEALTHY": "control", "Dyslexia": "dyslexia", "CHRONIC PAIN": "chronic pain", 
                            "MDD": "mdd", "Nan": "nan", "ADHD": "adhd", 
                            "ADHD/ASPERGER": "adhd/asperger", "PDD NOS/DYSLEXIA": "pdd nos/dyslexia", 
                            "PDD NOS": "pdd nos", "WHIPLASH": "whiplash", "ANXIETY": "anxiety",
                            "ADHD/DYSLEXIA": "adhd/dyslexia", "ASD": "asd", "TINNITUS": "tinnitus",
                            "OCD": "ocd", "Tinnitus": "tinnitus", "PDD NOS ": "pdd nos", "PANIC": "panic",
                            "MDD/ANXIETY": "mdd/anxiety", "MIGRAINE": "migraine", "PDD NOS/ANXIETY": "pdd nos/anxiety",
                            "PARKINSON": "parkinson",  "BIPOLAR": "bipolar",  "MDD/bipolar": "mdd/bipolar",
                            "DYSPRAXIA": "dyspraxia", "TINNITUS/MDD": "tinnitus/mdd", "ADHD/ASD/ANXIETY": "adhd/asd/anxiety",
                            "MDD/ADHD": "mdd/adhd",  "ADHD/PDD NOS": "adhd/pdd nos", "MDD/BIPOLAR": "mdd/bipolar",
                            "ASPERGER": "asperger", "ADHD/EPILEPSY": "adhd/epilepsy", "MDD/PAIN": "mdd/pain",
                            "PDD NOS/GTS": "pdd nos/gts",  "PDD NOS/ADHD": "pdd nos/adhd", "PDD NOS/ASD": "pdd nos/asd",
                            "TBI": "tbi", "ADHD/ANXIETY": "adhd/anxiety",  "ADHD/DYSLEXIA/DYSCALCULIA": "adhd/dyslexia/dyscalculia",
                            "ADHD/MDD": "adhd/mdd", "MDD/PANIC": "mdd/panic", "DEPERSONALIZATION": "depersonalization",
                            "MDD/TRAUMA": "mdd/trauma", "PTSD/ADHD": "ptsd/adhd",  "OCD/DPS": "ocd/dps","MDD/OCD": "mdd/ocd",
                            "MDD/TUMOR": "mdd/tumor", "ADHD/GTS": "adhd/gts", "OCD/MDD": "ocd/mdd", "CONVERSION DX": "conversion dx",
                            "ASD/ASPERGER": "asd/asperger", "MDD/ADHD/LYME": "mdd/adhd/lyme", "ADHD/OCD": "adhd/ocd",
                            "MSA-C": "msa-c", "OCD/ASD": "ocd/asd", "STROKE/PAIN": "stroke/pain",
                            "STROKE ": "stroke", "MDD/OCD/ADHD": "mdd/ocd/adhd",  "EPILEPSY/OCD": "epilepsy/ocd",
                            "ADHD ": "adhd", "INSOMNIA": "insomnia", "MDD/ADHD/ANOREXIA": "mdd/adhd/anorexia",
                            "MDD/ANXIETY/TINNITUS": "mdd/anxiety/tinnitus"}, "single_value":None})


file_dir3 = "/project/meganorm/Data/EEG_MIPDB/info/MIPDB_PublicFile.csv"
save_dir3 = "/project/meganorm/Data/EEG_MIPDB/EEG_BIDS/participants_bids.tsv"
make_demo_file_bids(file_dir3, 
                    save_dir3, 
                    0, 
                    1, 
                    {"col_name": "sex", "col_id": 2, "mapping": {1.0: "Male", 2.0: "Female"}, "single_value":None},
                    {"col_name": "eyes", "col_id": None, "mapping": None, "single_value":"eyes_closed"},
                    {"col_name": "diagnosis", "col_id": 37, "mapping": {
                        0: "control",
                        1: "unknown_1",
                        2: "unknown_2"}, "single_value":None})


# Merge features & covariates, define control & patients

In [6]:
### Data preparation for Normative Modeling
base_dirs = [values["base_dir"] for values in datasets.values()]
dataset_names = list(datasets.keys())

all_diagnosis = ['adhd', 'specific learning disorder', 'nonadherence to medical treatment', 'asd', 'anxiety disorder', 'communication disorder', 
                 'intellectual disability', 'depressive disorder', 'motor disorder', 'feeding and eating disorder', 'trauma and stressor related disorder', 
                 'disruptive, impulse control and conduct disorder', 'elimination disorder', 'bipolar and related disorder', 'obsessive compulsive and related disorder', 
                 'cannabis-related', 'neurocognitive disorder', 'schizophrenia spectrum and other psychotic disorder', 'other neurodevelopmental disorder', 'gender dysphoria', 
                 'burnout', 'dyslexia', 'chronic pain', 'mdd', 'ocd', 'parkinson', 'insomnia', 'unknown_1', 'unknown_2']

adhd_diagnosis = ['adhd']
parkinson_diagnosis = ['parkinson']
anxiety_diagnosis = ['anxiety disorder']
dyslexia_diaganosis = ['dyslexia']
learning_diagnosis = ['specific learning disorder']
autism_diagnosis = ['asd']
mdd_diagnosis = ['mdd']
depressive_diagnosis = ['depressive disorder']
ocd_diagnosis = ['ocd']
control = ['control']

merged_data, data_patient = merge_fidp_demo(base_dirs, features_dir, dataset_names, include_patients=False, diagnosis = all_diagnosis)
merged_data = merged_data.dropna()
data_patient = data_patient.dropna()
print(merged_data.shape)
print(data_patient.shape)

(394, 7)
(1925, 8)


# Subject information

In [ ]:
site_names = {
    0: "TDBrain",
    1: "MIPDB",
    2: "CMI1",
    3: "CMI2",
    4: "CMI3"
}

# Loop through sites and print info
for site, name in site_names.items():
    print(f"Dataset name: {name}")
    print(f"Number of controls: {merged_data[merged_data.site == site].shape[0]} participants")
    print(f"Number of patients: {data_patient[data_patient['site'] == site].shape[0]} participants")
    print(f"Mean age controls: {merged_data[merged_data.site == site]['age'].mean():.2f}")
    print(f"Mean age patients: {data_patient[data_patient.site == site]['age'].mean():.2f}")
    print(f"Max age controls: {merged_data[merged_data.site == site]['age'].max():.2f}")
    print(f"Max age patients: {data_patient[data_patient.site == site]['age'].max():.2f}")
    print(f"Min age controls: {merged_data[merged_data.site == site]['age'].min():.2f}")
    print(f"Min age patients: {data_patient[data_patient.site == site]['age'].min():.2f}")
    print(f"Std age controls: {merged_data[merged_data.site == site]['age'].std():.2f}")
    print(f"Std age patients: {data_patient[data_patient.site == site]['age'].std():.2f}")
    print(f"Female num control: {merged_data[np.logical_and(merged_data.site == site, merged_data.sex == 1)].shape[0]} participants")
    print(f"Female num patients: {data_patient[np.logical_and(data_patient.site == site, data_patient.sex == 1)].shape[0]} participants")
    print("-" * 50)  # Separator for better readability


In [ ]:
# plot age distribution for different sites and train/test/validation partitions #TRY 
path = "/home/meganorm-yverduyn/Results/Thesis_all_diagnosis/pics"
site_names = {"TDBrain","MIPDB","CMI1", "CMI2","CMI3", "CMI4", "CMI5"} 
plot_age_dist2(merged_data, site_names=site_names, save_path=path)

# Normative modeling 

In [7]:
subject_ids = merged_data.columns[0]
covariates = ["age"]
batch_effects = ["sex", "site"]
response_vars = "Adjusted_Canonical_Relative_PowerTheta_all,Adjusted_Canonical_Relative_PowerAlpha_all,Adjusted_Canonical_Relative_PowerBeta_all,Adjusted_Canonical_Relative_PowerGamma_all".split(',')

norm_data = NormData.from_dataframe(
    name="Thesis_refactored",
    dataframe=merged_data,
    covariates=covariates,
    batch_effects=batch_effects,
    response_vars=response_vars,
    subject_ids=subject_ids
)
train, test = norm_data.train_test_split(0.75)

Process: 289718 - 2025-03-12 13:46:09 - Dataset "Thesis_refactored" created.
    - 394 subjects
    - 1 covariates
    - 4 response variables
    - 2 batch effects:
    	sex (2)
	site (5)
    


In [14]:
#Configuration options for priors mu, sigma, epsilon and delta
mu = make_prior(
    linear=True,
    slope=make_prior(dist_name="Normal", dist_params=(0.0, 10.0)),
    intercept=make_prior(
        random=True,
        mu=make_prior(dist_name="Normal", dist_params=(1.0, 1.0)),
        sigma=make_prior(dist_name="Gamma", dist_params=(3.0, 1.0)),
    ),
    basis_function=BsplineBasisFunction(basis_column=0, nknots=5, degree=3),
)
sigma = make_prior(
    linear=True,
    slope=make_prior(dist_name="Normal", dist_params=(0.0, 2.0)),
    intercept=make_prior(dist_name="Normal", dist_params=(1.0, 1.0)),
    basis_function=BsplineBasisFunction(basis_column=0, nknots=5, degree=3),
    mapping="softplus",
    mapping_params=(0.0, 3.0),
)

epsilon = make_prior(
    linear = True,
    dist_name="Normal",
    dist_params=(0.0, 1.0),
)

delta = make_prior(
    linear = True, 
    dist_name="Normal",
    dist_params=(1.0, 1.0),
    mapping="softplus",
    mapping_params=(
        0.0,
        3.0,  # Scale for smoothness
        0.6,  # We need to provide a vertical shift as well, because the SHASH mapping goes a bit wild with low values for delta
    ),
)

# Specify the regression model
hbr_regression = HBR(
    name = "HBR", 
    cores=16,
    progressbar=True,
    draws=1500,
    tune = 500, 
    chains=4, 
    nuts_sampler="nutpie",
    likelihood= SHASHbLikelihood(mu, sigma, epsilon, delta)
)

In [15]:
save_dir = os.path.join(nm_processing_dir, "save_dir")
# Specify the normative model
hbr_model = NormativeModel(
    template_regression_model=hbr_regression,
    savemodel=True,
    evaluate_model=True,
    saveresults=True,
    saveplots=True,
    save_dir=save_dir,
    inscaler="none",
    outscaler="none",
)

In [ ]:
#run in parallel
sandbox_dir = os.path.join(nm_processing_dir, "runner_dir")
os.makedirs(sandbox_dir, exist_ok=True)

conda_env_path = "/project/meganorm/Software/Miniconda3/envs/mne"

runner = Runner(
    cross_validate=False,
    parallelize=False, 
    environment = conda_env_path,
    job_type="slurm",
    n_jobs= 4, 
    time_limit="00:120:00",
    log_dir=os.path.join(sandbox_dir, "log_dir"),
    temp_dir=os.path.join(sandbox_dir, "temp_dir"),
)

runner.fit_predict(hbr_model, train, test, observe=True)

2025-03-12 13:55:20 - Task ID created: fit_predict_Thesis_refactored_train__2025-03-12_13:55:20_423.414795
2025-03-12 13:55:20 - Temporary directory created:
	/home/meganorm-yverduyn/Results/Thesis_refactored/NM/Run_0/runner_dir/temp_dir/fit_predict_Thesis_refactored_train__2025-03-12_13:55:20_423.414795
2025-03-12 13:55:20 - Log directory created:
	/home/meganorm-yverduyn/Results/Thesis_refactored/NM/Run_0/runner_dir/log_dir/fit_predict_Thesis_refactored_train__2025-03-12_13:55:20_423.414795
2025-03-12 13:55:20 - Fitting models on 4 response variables.
2025-03-12 13:55:20 - Fitting model for Adjusted_Canonical_Relative_PowerTheta_all.


Progress,Draws,Divergences,Step Size,Gradients/Draw
,54,0,0.00,1023
,93,0,0.01,1023
,74,0,0.00,1023
,69,0,0.01,1023
